## Import Library


In [ ]:
a=pd.read_csv('IBM-3.csv')
a.drop(columns=['7_day_std',
       '7_SMA', '14_EMA', 'DMI_plus', 'DMI_minus', 'ATR', 'MACD',
       'MACD_Signal', 'stoch_k', 'stoch_d', 'close_change'], inplace=True)
a['Unnamed: 0'] = pd.to_datetime(a['Unnamed: 0'])
a.set_index('Unnamed: 0', inplace=True)
print(a)
a = a[a.index.isin(df.index)]

              open      high       low   close    volume
Unnamed: 0                                              
1999-11-01   98.50   98.8100   96.3700   96.75   9551800
1999-11-02   96.75   96.8100   93.6900   94.81  11105400
1999-11-03   95.87   95.9400   93.5000   94.37  10369100
1999-11-04   94.44   94.4400   90.0000   91.56  16697600
1999-11-05   92.75   92.9400   90.1900   90.25  13737600
...            ...       ...       ...     ...       ...
2024-11-04  207.65  207.7000  205.8000  206.32   2594119
2024-11-05  206.17  208.1150  205.5700  207.57   2441535
2024-11-06  213.48  214.3300  210.3700  213.60   3934386
2024-11-07  213.64  214.5199  211.9300  213.69   3675812
2024-11-08  214.16  216.7000  212.7809  213.72   3201038

[6297 rows x 5 columns]


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd
import numpy as np
from sklearn.metrics import explained_variance_score
dat=pd.read_csv('exog.csv')
diff=pd.read_csv('diff.csv')
df=dat.dropna()

df.iloc[:,0] = pd.to_datetime(df.iloc[:,0], errors='coerce')
df.set_index(df.columns[0], inplace=True)
df.drop(columns=df.columns[0], inplace=True)
diff.iloc[:,0] = pd.to_datetime(diff.iloc[:,0], errors='coerce')
diff.set_index(diff.columns[0], inplace=True)
merged_df = df.merge(diff, left_index=True, right_index=True, how='inner')
d=merged_df.drop('close',axis=1, inplace=False)
train_size = int(len(df) * 0.8)
trainset,testset = a['close'][:train_size],a['close'][train_size:]
exog = d
arimax = SARIMAX(a['close'], exog=exog, order=(1, 0, 0), seasonal_order=(0, 0, 0, 0))
model = arimax.fit()
prediction = model.predict(start=0, end=-1, exog=exog)
residuals = a['close'] - prediction
residuals_df = pd.DataFrame({'Residuals': residuals})

/usr/local/lib/python3.10/dist-packages/pandas/core/indexes/base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
<ipython-input-79-42c7bf8d24a1>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=df.columns[0], inplace=True)
/usr/local/lib/python3.10/dist-packages/pandas/core/indexes/base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
/usr/local

In [ ]:
train_size = int(len(residuals_df) * 0.8)
from torch.utils.data import DataLoader, TensorDataset
merged_df = df.merge(residuals_df, left_index=True, right_index=True, how='inner')
d=merged_df.drop('Residuals',axis=1, inplace=False)
X_train,X_test = d[:train_size], d[train_size:]
Y_train,Y_test = merged_df['Residuals'][:train_size], merged_df['Residuals'][train_size:]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)  # Convert DataFrame to tensor
Y_train_tensor = torch.tensor(Y_train.values, dtype=torch.float32)  # Convert Series to tensor


In [ ]:
from ray.tune import choice
from ray.tune import uniform
from ray.tune import uniform

In [ ]:
from ray import train #import train to get train.report instead of keras integrations

In [ ]:
!pip install ray[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 3

In [ ]:

import numpy as np
#wrapped to take config dictionary
def train_model(config):
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import DataLoader, TensorDataset

    class FC2Layer(nn.Module):
      def __init__(self, input_size, n_hidden, output_size, dropout_prob, activation):
        super(FC2Layer, self).__init__()
        self.input_size = input_size
        self.dropout_prob = dropout_prob
        if activation =='relu':
            self.activation = nn.ReLU()
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        else :
            self.activation = nn.ReLU()
            print("Activation function not implemented, using default ReLU")
        self.network = nn.Sequential(
            nn.Linear(input_size, n_hidden),
            self.activation,
            nn.Dropout(p=dropout_prob),
            nn.Linear(n_hidden, n_hidden),
            self.activation,
            nn.Dropout(p=dropout_prob),
            nn.Linear(n_hidden, output_size)
        )

      def forward(self, x):
          return self.network(x)


    # Synthetic data for example
    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)  # Convert DataFrame to tensor
    Y_train_tensor = torch.tensor(Y_train.values, dtype=torch.float32)


    if len(Y_train_tensor.shape) == 1:
      Y_train_tensor = Y_train_tensor.unsqueeze(1)


    train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
    Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.float32)
    if len(Y_test_tensor.shape) == 1:
      Y_test_tensor = Y_test_tensor.unsqueeze(1)
    test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

    # Initialize the model
    model = FC2Layer(
        input_size=4,
        n_hidden=int(config["n_hidden"]),
        output_size=1,
        dropout_prob=config["dropout_prob"],
        activation=config['activation']
    )

    # Apply weight initializer
    if config["initializer"] == "xavier":
        def xavier_init(m):
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        model.apply(xavier_init)
    elif config["initializer"] == "normal":
        def normal_init(m):
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0.0, std=0.02)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        model.apply(normal_init)

    # Select optimizer
    if config["optimizer"] == "adam":
        optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
    elif config["optimizer"] == "sgd":
        optimizer = optim.SGD(model.parameters(), lr=config["learning_rate"], momentum=0.9)
    elif config["optimizer"] == "rmsprop":
        optimizer = optim.RMSprop(model.parameters(), lr=config["learning_rate"])

    # Loss function
    criterion = nn.MSELoss()

    # Training loop
    model.train()
    for epoch in range(int(config["num_epochs"])):
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    # Testing loop
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
    mse_loss = test_loss / len(test_loader)

    # Report result to Ray Tune
    train.report({"MSE": mse_loss})

# Define search space
search_space = {
    "n_hidden": np.random.choice([2, 5, 10]),
    "dropout_prob": np.random.uniform(0.1, 0.5),
    "learning_rate": np.random.uniform(0.001, 0.1),
    "batch_size": np.random.randint(2,10),
    "num_epochs": np.random.choice([2, 5, 10]),
    "optimizer": np.random.choice(["adam", "sgd", "rmsprop"]),
    "initializer": np.random.choice(["xavier", "normal"]),
    'activation':np.random.choice(["tanh", "sigmoid", 'relu'])
}

# Run the search
analysis_grid = tune.run(
    train_model,
    name="exp",
    metric="MSE",
    mode="min",
    stop={"MSE": 2},
    resources_per_trial={"cpu": 1, "gpu": 1},
    config=search_space,
)

# Best configuration
best_grid = analysis_grid.best_config
print("Best configuration:", best_grid)


+--------------------------------------------------------+
| Configuration for experiment     exp                   |
+--------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator |
| Scheduler                        FIFOScheduler         |
| Number of trials                 1                     |
+--------------------------------------------------------+

View detailed results here: /root/ray_results/exp
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-12-13_19-35-39_157175_590/artifacts/2024-12-13_21-44-14/exp/driver_artifacts`

Trial status: 1 PENDING
Current time: 2024-12-13 21:44:14. Total running time: 0s
Logical resource usage: 1.0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+------------------------------------+
| Trial name                status   |
+------------------------------------+
| train_model_658fa_00000   PENDING  |
+------------------------------------+

Trial tra

2024-12-13 21:44:27,674	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/exp' in 0.0046s.



Trial train_model_658fa_00000 finished iteration 1 at 2024-12-13 21:44:27. Total running time: 12s
+--------------------------------------------------+
| Trial train_model_658fa_00000 result             |
+--------------------------------------------------+
| checkpoint_dir_name                              |
| time_this_iter_s                         11.2884 |
| time_total_s                             11.2884 |
| training_iteration                             1 |
| MSE                                      4.82523 |
+--------------------------------------------------+

Trial train_model_658fa_00000 completed after 1 iterations at 2024-12-13 21:44:27. Total running time: 12s

Trial status: 1 TERMINATED
Current time: 2024-12-13 21:44:27. Total running time: 12s
Logical resource usage: 1.0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
Current best trial: 658fa_00000 with MSE=4.825228982333895 and params={'n_hidden': 5, 'dropout_prob': 0.21307617072616686, 'learning_rate': 0.035674301

In [ ]:
config={"n_hidden": 2,
    "dropout_prob": .1,
    "learning_rate": .001,
    "batch_size": 4,
    "num_epochs": 5,
    "optimizer": 'adam',
    "initializer": 'xavier',
    "activation":'tanh'}
train_model(config)

In [ ]:
hyperband_scheduler = HyperBandScheduler(
     # Maximize the mean_accuracy metric
    max_t=100,  # Maximum resource allocation (e.g., epochs)
    # Minimum resource allocation before stopping
)

# Run the experiment with HyperBand
analysis_hyperband = tune.run(
    train_model,
    name="hyperband_exp",
    metric="MSE",
    mode="min",
    resources_per_trial={
         "cpu": 1, "gpu": 1
    },
    config=search_space,
    scheduler=hyperband_scheduler  # Add the HyperBand scheduler
)

best_hyperband = analysis_hyperband.best_config


+--------------------------------------------------------+
| Configuration for experiment     hyperband_exp         |
+--------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator |
| Scheduler                        HyperBandScheduler    |
| Number of trials                 1                     |
+--------------------------------------------------------+

View detailed results here: /root/ray_results/hyperband_exp
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-12-13_19-35-39_157175_590/artifacts/2024-12-13_21-56-07/hyperband_exp/driver_artifacts`

Trial status: 1 PENDING
Current time: 2024-12-13 21:56:07. Total running time: 0s
Logical resource usage: 1.0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+------------------------------------+
| Trial name                status   |
+------------------------------------+
| train_model_0ea08_00000   PENDING  |
+----------------------------

2024-12-13 21:56:22,327	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/hyperband_exp' in 0.0044s.



Trial train_model_0ea08_00000 finished iteration 1 at 2024-12-13 21:56:22. Total running time: 14s
+--------------------------------------------------+
| Trial train_model_0ea08_00000 result             |
+--------------------------------------------------+
| checkpoint_dir_name                              |
| time_this_iter_s                         12.7985 |
| time_total_s                             12.7985 |
| training_iteration                             1 |
| MSE                                      4.83112 |
+--------------------------------------------------+

Trial train_model_0ea08_00000 completed after 1 iterations at 2024-12-13 21:56:22. Total running time: 14s

Trial status: 1 TERMINATED
Current time: 2024-12-13 21:56:22. Total running time: 14s
Logical resource usage: 1.0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
Current best trial: 0ea08_00000 with MSE=4.8311213830226585 and params={'n_hidden': 5, 'dropout_prob': 0.21307617072616686, 'learning_rate': 0.03567430

In [ ]:
print(best_hyperband)

{'n_hidden': 5, 'dropout_prob': 0.21307617072616686, 'learning_rate': 0.03567430187528342, 'batch_size': 4, 'num_epochs': 5, 'optimizer': 'adam', 'initializer': 'xavier', 'activation': 'tanh'}
